In [75]:
import requests # to get the HTML
from bs4 import BeautifulSoup # to get parse the text
import numpy as np
import pandas as pd
import tqdm
from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works
driver = webdriver.Chrome(options=chrome_options)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
pd.options.display.max_colwidth = 200

In [3]:
base_url = 'https://www.jobs.bg/'
url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
#      https://www.jobs.bg/front_job_search.php?frompage=15&add_sh=1&from_hp=1&term=#paging

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
def get_pages(soup):
    """Get the total number of job offerings and the total number of pages.
    output: total number of pages
    """
    total_job_number = soup.find('td',
                            attrs={'style':
                                  'height:25px;width:220px;font-weight:500;padding-bottom:5px;'}).text.split()[-1]
    total_job_number= int(total_job_number)
    total_pages = int(total_job_number/15)+1 #round up
    return total_pages

In [39]:
# total_pages = get_pages(soup)
total_pages = 30
results =[]

#for page in tqdm.tqdm(range(total_pages)):
for page in tqdm.tqdm(range(0, 1)):

    url = f'https://www.jobs.bg/front_job_search.php?frompage={page*15}&add_sh=1&from_hp=1&term=#paging'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})
    
    for i in range(0, 45, 3):
        text_info_elem = job_infos[i]
        job_logo_elem = job_infos[i+1]
        job_company_elem = job_infos[i+2]
        
        position_id = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href'].lstrip('job/')
        position_url = base_url+text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href']
        company_name = job_company_elem.text.strip()

        
        position_r = requests.get(position_url)
        position_soup = BeautifulSoup(position_r.text, 'html.parser')
        
        position_views = position_soup.find('span', string = 'Разглеждания:').parent.text
        
        iframe_elem = position_soup.find('iframe', attrs = {'id':'customJobIframe'})
        raw_html_elem = position_soup.find('td', attrs = {'class':'jobTitle'})
        if iframe_elem:
            iframe_url = iframe_elem.attrs['src']
            iframe_r = requests.get(iframe_url)
            iframe_soup = BeautifulSoup(iframe_r.text, 'html.parser')
            job_text = iframe_soup.text.strip()
        elif raw_html_elem: 
            job_text=raw_html_elem.parent.parent.text.strip()
        else:
            job_text = np.nan
        
        job_date = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'}).text.strip()
        if job_date == 'днес':
            job_date = date.today().strftime("%d.%m.%y")
        elif job_date == 'вчера':
            job_date = (date.today() - timedelta(days=1)).strftime("%d.%m.%y")

        rating_elem = text_info_elem.find('span',attrs={'class':'iconed'})

        if rating_elem:
            rataing_elem_styles = rating_elem["style"].split(";")
            rating_elem_color_styles = [x for x in rataing_elem_styles if "color:" in x]
            rating_color = rating_elem_color_styles[0].split(":")[1]
            rating_full = rating_elem.text.strip().count("\uf4b3")
            rating_half = (rating_elem.text.strip().count("\uf4b1"))/2
            rating_final = rating_full + rating_half
        else:
            rating_color = np.nan
            rating = np.nan
       
        position = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'}).text.strip()
        benefits = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'}).text.strip()
        
        tech_list = []
        for a in text_info_elem.find_all('img'):
            tech = a.attrs['alt']
            tech_list.append(tech)
        tech_list = ", ".join(tech_list)
        results.append([position_id,position_url,company_name,position_views,job_text,job_date, rating_final, rating_color, position, benefits, tech_list])

100%|██████████| 1/1 [00:04<00:00,  4.73s/it]


In [40]:
jobs_df = pd.DataFrame(results, columns=["Position_Id","Position_URL","Company Name","Position Views","Job Description","Date Posted", 
                                         "Position_Rating","Rating Color", "Position", "Benefits", "Technologies"])
jobs_df

,Position_Id,Position_URL,Company Name,Position Views,Job Description,Date Posted,Position_Rating,Rating Color,Position,Benefits,Technologies
0,5665200,https://www.jobs.bg/job/5665200,TELUS INTERNATIONAL EUROPE / CALLPOINT NEW EUROPE,Разглеждания:,Our Partnership with Airbnb gives you a unique opportunity to become a part of the global team that focuses its attention on positively impacting the experiences that customers have when interacti...,07.05.21,5.0,#66c1ff,French Speaking Travel Experience Advisor for Airbnb,София; Дистанционно интервю,"Английски, Френски"
1,5680478,https://www.jobs.bg/job/5680478,Астреа Рикрутмънт ЕООД,Разглеждания:,"Human factor is the most crucial aspect for the success of every organization.Astrea creates solutions, while at the same time puts into this process uniqueness, creativity,flexibility, energy and...",07.05.21,5.0,#66c1ff,"British English Customer Support Representatives with English for international company (4,6,8h)",София,Английски
2,5680473,https://www.jobs.bg/job/5680473,Астреа Рикрутмънт ЕООД,Разглеждания:,"Астреа е създадена през 2007 година с една основна мисия – да бъдем различни.Превърнахме това виждане в основен двигател на нашите усилия да сме винаги на разположение, когато клиентите и партньор...",07.05.21,5.0,#66c1ff,"Консултант обслужване на потребители с чешки, словашки, немски, френски, италиански",София; Заплата от 1800 до 2300 BGN (Бруто),"Английски, Немски, Словашки, Френски, Чешки"
3,5680468,https://www.jobs.bg/job/5680468,Астреа Рикрутмънт ЕООД,Разглеждания:,Astrea Human Resources was found in 2007 with one principal mission – to be different.We transformed this vision into a key driver of our efforts to be always available when ourclients or partners...,07.05.21,5.0,#66c1ff,"Social Media Content Moderator with Armenian, Finnish, French, German & English",София; Заплата от 1800 до 3000 BGN (Бруто),"Английски, Немски, Фински, Френски"
4,5676544,https://www.jobs.bg/job/5676544,EURODRIVE OOD,Разглеждания:,"ИСКАШ ДА РАБОТИШ КАТО МЕЖДУНАРОДЕН ШОФЬОР, НО НЯМАШ ОПИТ!!ЕЛА ПРИ НАС!! ЮРОДРАЙВ ТИ ПРЕДЛАГА ТУК И СЕГА: ЩЕ ТЕ ОБУЧИМ и на теория и на практика. Ще ти изкараме необходимите документи, ще те обучим...",07.05.21,5.0,#66c1ff,ВОДАЧИ БЕЗ ОПИТ ЗА МЕЖДУНАРОДНИ ШОФЬОРИ,София; Заплата от 2100 до 2600 EUR (Нето),
5,5680461,https://www.jobs.bg/job/5680461,МАГ ООД,Разглеждания:,КОНСТРУКТОР-ЧЕРТОЖНИК ЗА МАШИНОСТРОИТЕЛНО ПРЕДПРИЯТИЕ \n\n\n\n\n\t\t\t\t\t\t \t\t\t\t\t\t \tОсновни задължения:\r\n- Проектира и/или Изчертава метални изделия и ма...,07.05.21,5.0,#66c1ff,КОНСТРУКТОР-ЧЕРТОЖНИК ЗА МАШИНОСТРОИТЕЛНО ПРЕДПРИЯТИЕ,Бобов дол; Заплата от 1200 до 2000 BGN (Нето),
6,5680409,https://www.jobs.bg/job/5680409,Кауфланд България ЕООД енд Ко КД,Разглеждания:,"Старши специалист Подбор,\r\n \nЧовешки ресурси\r\n \nДай успешен старт на кариерата си в международна търговска компания, която залага на резултати, динамика и коректност. Със с...",07.05.21,5.0,#66c1ff,"Старши специалист Подбор, Човешки ресурси",София,
7,5680472,https://www.jobs.bg/job/5680472,Трансатлантик Лоджистикс ЕООД,Разглеждания:,"Диспечер \n\n\n\n\n\t\t\t\t\t\t \t\t\t\t\t\t \t​​​​​Night shift dispatcher​\n\r\nJob Summary:\n\r\nUnder routine supervision, this position is responsible for scheduling...",07.05.21,5.0,#66c1ff,Диспечер,Пловдив; Заплата от 1000 до 1800 BGN (Нето),
8,5680436,https://www.jobs.bg/job/5680436,Агенция за събиране на вземания ЕАД,Разглеждания:,"Станете новото свежо попълнение към нашия Кол-център на позиция: Агент събиране на вземания!\n\n""Агенция за събиране на вземания"" ЕАД е бързо развиваща се компания, част от B2Holding - международн...",07.05.21,5.0,#66c1ff,Кол-център Агент,Плевен,
9,5680444,https://www.jobs.bg/job/5680444,MentorMate Bulgaria Ltd.,Разглеждания:,".NET Team Lead\r\n \n\nNote: We offer flexible working models for all open positions. You can choose from three options: a combination of work from home and office, work only from home, o...",07.05.21,5.0,#66c1ff,.NET Team Lead,София; Възм

In [26]:
job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})

for i in range(0, 45, 3):
    text_info_elem = job_infos[i]
    job_logo_elem = job_infos[i+1]
    job_company_elem = job_infos[i+2]
   
print(job_company_elem.text.strip())


BALIS MANAGEMENT EOOD



In [76]:
driver.get(position_url)
html = driver.execute_script("return document.documentElement.outerHTML")
position_soup = BeautifulSoup(html, 'html.parser')

In [77]:
#position_r = requests.get(position_url)
#position_soup = BeautifulSoup(position_r.text, 'html.parser')

position_views = position_soup.find('span', string = 'Разглеждания:').parent

In [78]:
position_views.text

'Разглеждания: 193'